In [ ]:
# Copyright (c) MONAI Consortium
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#     http://www.apache.org/licenses/LICENSE-2.0
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Denoising Diffusion Probabilistic Model on 3D data

This tutorial illustrates how to use MONAI for training a denoising diffusion probabilistic model (DDPM)[1] to create synthetic 3D images.

[1] - [Ho et al. "Denoising Diffusion Probabilistic Models"](https://arxiv.org/abs/2006.11239)


## Setup environment

In [ ]:
!python -c "import monai" || pip install -q "monai-weekly[nibabel, tqdm]"
!python -c "import matplotlib" || pip install -q matplotlib
%matplotlib inline

## Setup imports

In [ ]:
import os
import tempfile
import time
import glob

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn.functional as F
from monai.apps import DecathlonDataset
from monai.config import print_config
from monai.data import DataLoader
from monai.transforms import (
    EnsureChannelFirstd,
    CenterSpatialCropd,
    Compose,
    Lambdad,
    LoadImaged,
    Resized,
    ScaleIntensityd,
)
from monai.utils import set_determinism
from torch.cuda.amp import GradScaler, autocast
from tqdm import tqdm

from generative.inferers import DiffusionInferer
from generative.networks.nets import DiffusionModelUNet
from generative.networks.schedulers import DDPMScheduler, DDIMScheduler
print_config()

## Setup data directory

You can specify a directory with the MONAI_DATA_DIRECTORY environment variable.

This allows you to save results and reuse downloads.

If not specified a temporary directory will be used.

In [ ]:
root_dir = "DATASET"
assert os.path.isdir(root_dir), "Check PI-CAI path"
print(root_dir)


## Set deterministic training for reproducibility

In [ ]:
set_determinism(42)

## Setup Decathlon Dataset and training and validation data loaders

In this tutorial, we will use the 3D T1 weighted brain images from the [2016 and 2017 Brain Tumor Segmentation (BraTS) challenges](https://www.med.upenn.edu/sbia/brats2017/data.html). This dataset can be easily downloaded using the [DecathlonDataset](https://docs.monai.io/en/stable/apps.html#monai.apps.DecathlonDataset) from MONAI (`task="Task01_BrainTumour"`). To load the training and validation images, we are using the `data_transform` transformations that are responsible for the following:

1. `LoadImaged`:  Loads the brain images from files.
2. `Lambdad`: Choose channel 1 of the image, which is the T1-weighted image.
3. `EnsureChannelFirstd`: Add the channel dimension of the input data.
4. `ScaleIntensityd`: Apply a min-max scaling in the intensity values of each image to be in the `[0, 1]` range.
5. `CenterSpatialCropd`: Crop the background of the images using a roi of size `[160, 200, 155]`.
6. `Resized`: Resize the images to a volume with size `[32, 40, 32]`.

For the data loader, we are using mini-batches of 8 images, which consumes about 21GB of GPU memory during training. Please, reduce this value to run on smaller GPUs.

In [ ]:
image_paths = sorted(glob.glob(f"{root_dir}/*/*_t2w.mha"))
print(f"Found {len(image_paths)} volumes.")

In [ ]:
import os, glob
import SimpleITK as sitk
import torch
from monai.transforms import (
    Transform,
    Orientationd,
    Spacingd,
    CropForegroundd,
    RandSpatialCropd,
    ToTensord,
    LoadImaged,
    ScaleIntensityRangePercentilesd,
    EnsureChannelFirstd,
    SpatialPadd,
    EnsureTyped,
)
from monai.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from pathlib import Path
import logging

def filter_valid_3d(paths, min_depth=3):
    """
    Returns only those .mha files that load as true 3-D volumes.
    A file is dropped if:
      • img.GetDimension() < 3   –> actually 2-D
      • min(img.GetSize()) < min_depth –> depth too small
    """
    keep, drop = [], []
    for p in paths:
        img = sitk.ReadImage(str(p))
        dim = img.GetDimension()
        size = img.GetSize()
        if dim == 3 and min(size) >= min_depth:
            keep.append(p)
        else:
            drop.append((p, dim, size))
    print(f"kept {len(keep)} volumes and dropped {len(drop)} bad files")
    if drop:
        print("first few dropped:", *(Path(d[0]).name for d in drop[:5]))
    return keep
    
root_dir    = "DATASET"  
all_paths = sorted(Path(root_dir).glob("*/*_t2w.mha"))
good_paths = filter_valid_3d(all_paths)

image_paths = sorted(glob.glob(f"{root_dir}/*/*_t2w.mha"))
train_paths, val_paths = train_test_split(
    image_paths, test_size=0.3, random_state=42
)

train_dicts = [{"image": p} for p in train_paths]
val_dicts   = [{"image": p} for p in val_paths]

pixdim     = (0.5, 0.5, 1.0)
margin     = (8, 32, 32)       
patch_size = (64, 128, 128)    

class LogIntensityStats(Transform):
    def __init__(self, lower: float, upper: float):
        self.lower_pct = lower
        self.upper_pct = upper

    def __call__(self, data):
        img = data["image"].numpy().flatten()
        low_val  = np.percentile(img, self.lower_pct * 100)
        high_val = np.percentile(img, self.upper_pct * 100)
        frac_low  = (img < low_val).mean()
        frac_high = (img > high_val).mean()
        new_min, new_max = img.min(), img.max()

        logging.info(
            f"[IntensityStats] pct window {self.lower_pct*100:.1f}-{self.upper_pct*100:.1f} → "
            f"[{low_val:.3f},{high_val:.3f}], clipped voxels: "
            f"{frac_low*100:.1f}% low, {frac_high*100:.1f}% high, "
            f"post-scaled range [{new_min:.3f},{new_max:.3f}]"
        )
        return data

transforms = Compose([
    LoadImaged(keys="image", image_only=False, reader="ITKReader"),
    EnsureTyped(keys="image", track_meta=True),
    EnsureChannelFirstd(keys="image"),
    Orientationd(keys="image", axcodes="RAS"),
    Spacingd(keys="image", pixdim=pixdim, mode="bilinear"),
    CropForegroundd(
        keys="image", source_key="image",
        margin=margin, select_fn=lambda x: x > 0, k_divisible=8
    ),
    ScaleIntensityRangePercentilesd(
        keys="image", lower=0.5, upper=99.5,
        b_min=-1.0, b_max=1.0, clip=True
    ),
    LogIntensityStats(lower=0.005, upper=0.995),
    SpatialPadd(keys="image", spatial_size=patch_size, mode="edge"),
    RandSpatialCropd(keys="image", roi_size=patch_size, random_size=False),
    ToTensord(keys="image"),
])

def assert_pipeline_ok(ds, atol_rot=0.1):  # Increased tolerance to 0.1
    """
    Validates the preprocessing pipeline output.
    
    Args:
        ds: Dataset to test
        atol_rot: Rotation tolerance. Medical images often have small residual
                 rotations after resampling, so 0.1 is more realistic than 1e-4
    """
    s = ds[0]
    aff = torch.as_tensor(s["image_meta_dict"]["affine"])
    A = aff[:3, :3]
    
    # Check voxel sizes
    vs = torch.linalg.norm(A, dim=0)
    expected_vs = torch.tensor(pixdim)
    vs_error = (vs - expected_vs).abs().max().item()
    print(f"Voxel sizes: {[f'{v:.3f}' for v in vs]} (expected: {[f'{v:.3f}' for v in expected_vs]})")
    print(f"Max voxel size error: {vs_error:.6f}")
    
    # Check rotation
    R = A / vs
    rot_err = (R.abs() - torch.eye(3)).abs().max().item()
    print(f"Max rotation error: {rot_err:.6f}")
    
    # Shape and intensity checks
    img = s["image"]
    print(f"Patch shape: {tuple(img.shape)}")
    print(f"Intensity range: [{img.min():.3f}, {img.max():.3f}]")
    
    # Assertions with helpful error messages
    if vs_error > 0.1:  # Allow 10% voxel size error
        print(f" Warning: Voxel size error {vs_error:.3f} > 0.1")
    
    if rot_err > atol_rot:
        print(f" Warning: Rotation error {rot_err:.3f} > {atol_rot}")
    else:
        print("Orientation within tolerance")
    
    # Intensity range check
    assert -1.1 <= img.min() <= -0.8, f"Min intensity {img.min()} not in expected range"
    assert 0.8 <= img.max() <= 1.1, f"Max intensity {img.max()} not in expected range"
    print("Intensity range OK")
    
    # Shape check
    assert tuple(img.shape) == (1, *patch_size), f"Shape {tuple(img.shape)} != expected {(1, *patch_size)}"
    print("Shape OK")
    
    return True

# Create datasets
print("Creating datasets with fixed transforms...")
train_ds = Dataset(data=train_dicts, transform=transforms)
val_ds = Dataset(data=val_dicts, transform=transforms)

# Test the pipeline
print("\nTesting pipeline...")
assert_pipeline_ok(train_ds)

# Create data loaders
train_loader = DataLoader(
    train_ds, batch_size=1, shuffle=True,
    num_workers=4, pin_memory=True, persistent_workers=True
)

val_loader = DataLoader(
    val_ds, batch_size=1, shuffle=False,
    num_workers=4, pin_memory=True, persistent_workers=True
)

print("\n Pipeline setup complete!")
print(f"Training samples: {len(train_ds)}")
print(f"Validation samples: {len(val_ds)}")

print("\nChecking first 3 samples...")
for i in range(min(3, len(train_ds))):
    try:
        sample = train_ds[i]
        img = sample["image"]
        aff = sample["image_meta_dict"]["affine"]
        
        print(f"Sample {i}: shape={tuple(img.shape)}, "
              f"range=[{img.min():.2f}, {img.max():.2f}], "
              f"spacing={torch.linalg.norm(aff[:3,:3], dim=0).numpy()}")
    except Exception as e:
        print(f"Sample {i}: ERROR - {e}")

In [ ]:
import torch, torch.nn as nn, torch.nn.functional as F
from torch.optim import Adam, AdamW
from torch.amp import GradScaler, autocast
from tqdm import tqdm
from generative.networks.nets import DiffusionModelUNet
from generative.inferers import DiffusionInferer
from generative.networks.schedulers import DDPMScheduler
import matplotlib.pyplot as plt
import numpy as np
import math

device = torch.device("cuda")

class Small3DVAE(nn.Module):
    def __init__(self, in_ch=1, latent_ch=32, beta=1.0):
        super().__init__()
        self.conv1 = nn.Sequential(
            nn.Conv3d(in_ch, 32, 4, 2, 1), nn.InstanceNorm3d(32), nn.ReLU(inplace=True))
        self.conv2 = nn.Sequential(
            nn.Conv3d(32, 64, 4, 2, 1), nn.InstanceNorm3d(64), nn.ReLU(inplace=True))
        self.conv_mu    = nn.Conv3d(64, latent_ch, 4, 2, 1)
        self.conv_logvar= nn.Conv3d(64, latent_ch, 4, 2, 1)
        self.dec = nn.Sequential(
            nn.Upsample(scale_factor=2, mode="trilinear", align_corners=False),
            nn.Conv3d(latent_ch, 64, 3, padding=1), nn.InstanceNorm3d(64), nn.ReLU(inplace=True),
            nn.Upsample(scale_factor=2, mode="trilinear", align_corners=False),
            nn.Conv3d(64, 32, 3, padding=1), nn.InstanceNorm3d(32), nn.ReLU(inplace=True),
            nn.Upsample(scale_factor=2, mode="trilinear", align_corners=False),
            nn.Conv3d(32, in_ch, 3, padding=1), nn.Tanh()
        )
        self.beta = beta

    def encode(self, x):
        h = self.conv1(x)
        h = self.conv2(h)
        mu     = self.conv_mu(h)
        logvar = self.conv_logvar(h)
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        return self.dec(z)

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        recon = self.decode(z)
        # compute KL = 0.5 * sum(μ² + σ² - logσ² - 1)
        kl = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())  
        return recon, mu, logvar, kl


vae = Small3DVAE(in_ch=1, latent_ch=32).to(device)

# Shape & grad‐flow sanity checks
x_test = torch.randn(2, 1, *patch_size, device=device)
recon_test, mu_test, logvar_test, kl_test = vae(x_test)
z_test = vae.reparameterize(mu_test, logvar_test)
assert recon_test.shape == x_test.shape, f"VAE recon {recon_test.shape} != {x_test.shape}"
assert z_test.shape == (2, 32, *(s//8 for s in patch_size)), f"latent shape wrong: {z_test.shape}"

# 3) Train VAE
vae_opt    = Adam(vae.parameters(), lr=1e-3)
vae_epochs = 5
scaler     = GradScaler()
vae_losses = []

for ep in range(vae_epochs):
    vae.train()
    total_loss = 0
    pbar = tqdm(train_loader, desc=f"VAE ep{ep}", ncols=70)
    for batch in pbar:
        x = batch["image"].to(device)
        vae_opt.zero_grad(set_to_none=True)
        with autocast(device_type="cuda"):
            recon, mu, logvar, kl = vae(x)
            rec_loss = F.mse_loss(recon, x)
            kl_loss  = kl / (x.numel())
            loss     = rec_loss + vae.beta * kl_loss
        scaler.scale(loss).backward()
        scaler.step(vae_opt)
        scaler.update()
        total_loss += loss.item()
        pbar.set_postfix({"loss": total_loss/(pbar.n or 1)})
        vae_losses.append(loss.item())
    print(f"→ VAE epoch {ep} avg loss: {total_loss/len(train_loader):.4f}")

# freeze VAE
for p in vae.parameters(): p.requires_grad = False
vae.eval()

# VAE reconstruction quality on a val batch 
val_batch = next(iter(val_loader))
x_val = val_batch["image"].to(device)
with torch.no_grad():
    recon_val, mu_val, logvar_val, kl_val = vae(x_val)

mse_val = F.mse_loss(recon_val, x_val).item()
psnr_val = 10 * math.log10(1.0 / mse_val) if mse_val > 0 else float('inf')
print(f"VAE val MSE: {mse_val:.4f}, PSNR: {psnr_val:.2f} dB")

#  Build DDPM on latent space
latent_ch   = 32
latent_size = (s//8 for s in patch_size)
latent_size = tuple(latent_size)

ddpm = DiffusionModelUNet(
    spatial_dims=3,
    in_channels=latent_ch,
    out_channels=latent_ch,
    num_channels=[32,64,128],
    attention_levels=[False,False,False],
    num_head_channels=[0,0,latent_ch],
    num_res_blocks=2,
).to(device)

scheduler = DDPMScheduler(num_train_timesteps=1000, schedule="linear_beta")
inferer   = DiffusionInferer(scheduler=scheduler)
ddpm_opt  = AdamW(ddpm.parameters(), lr=2e-4, weight_decay=1e-5)
ddpm_losses = []

# quick forward‐shape check
with torch.no_grad():
    pred_test = inferer(
        inputs=z_test, diffusion_model=ddpm,
        noise=torch.randn_like(z_test),
        timesteps=torch.randint(0, scheduler.num_train_timesteps, (z_test.shape[0],), device=device)
    )
assert pred_test.shape == z_test.shape, f"DDPM out {pred_test.shape} != {z_test.shape}"

# Train DDPM on latents
n_epochs = 150
val_interval = 25

for epoch in range(n_epochs):
    ddpm.train()
    epoch_loss = 0
    pbar = tqdm(train_loader, desc=f"DDPM ep{epoch}", ncols=70)
    for batch in pbar:
        x = batch["image"].to(device)
        with torch.no_grad():
            mu, logvar = vae.encode(x)
            z = vae.reparameterize(mu, logvar)  # B×32×8×16×16
        ddpm_opt.zero_grad(set_to_none=True)
        with autocast(device_type="cuda"):
            noise = torch.randn_like(z)
            t = torch.randint(0, scheduler.num_train_timesteps,
                              (z.shape[0],), device=device).long()
            pred = inferer(inputs=z, diffusion_model=ddpm,
                           noise=noise, timesteps=t)
            loss = F.mse_loss(pred, noise)
        scaler.scale(loss).backward()
        scaler.step(ddpm_opt)
        scaler.update()
        epoch_loss += loss.item()
        ddpm_losses.append(loss.item())
        pbar.set_postfix({"loss": epoch_loss/(pbar.n or 1)})
    print(f"→ DDPM epoch {epoch} avg loss: {epoch_loss/len(train_loader):.4f}")

    if epoch % val_interval == 0:
        ddpm.eval()
        # qualitative sample
        z0 = torch.randn((1, latent_ch, *latent_size), device=device)
        scheduler.set_timesteps(num_inference_steps=250)
        with torch.no_grad(), autocast(device_type="cuda"):
            z_samp = inferer.sample(input_noise=z0, diffusion_model=ddpm, scheduler=scheduler)
            out = vae.decode(z_samp).cpu()[0,0]
        plt.figure(); plt.imshow(out[patch_size[0]//2], cmap="gray"); plt.axis("off")
        plt.title(f"Sample @ epoch {epoch}"); plt.show()
        
        # diversity metric
        zs = []
        for _ in range(4):
            z0 = torch.randn((1, latent_ch, *latent_size), device=device)
            z_samp = inferer.sample(input_noise=z0, diffusion_model=ddpm, scheduler=scheduler)
            zs.append(z_samp.flatten().cpu().numpy())
        dists = []
        for i in range(len(zs)):
            for j in range(i+1, len(zs)):
                dists.append(np.mean((zs[i]-zs[j])**2))
        print(f"Avg pairwise latent MSE: {np.mean(dists):.4e}")

        ddpm.train()

# Final sample & display
ddpm.eval()
z0 = torch.randn((1, latent_ch, *latent_size), device=device)
scheduler.set_timesteps(num_inference_steps=250)
with torch.no_grad(), autocast(device_type="cuda"):
    z_sample   = inferer.sample(input_noise=z0, diffusion_model=ddpm, scheduler=scheduler)
    patch_synth = vae.decode(z_sample).cpu()   # [1,1,Z,Y,X]

slice_idx = patch_size[0] // 2
plt.figure(figsize=(4,4))
plt.imshow(patch_synth[0, 0, slice_idx, :, :], cmap="gray")
plt.axis("off")
plt.title("Final VAE+DDPM Sample")
plt.show()


### Visualization of the training images

In [ ]:
import random
import matplotlib.pyplot as plt
from monai.transforms import (
    Compose,
    Orientationd,
    Spacingd,
    ScaleIntensityRanged,
    DivisiblePadd,
    ToTensord,
)
from monai.data import PersistentDataset, DataLoader

full_transforms = Compose([
    LoadMHAwithSimpleITK(),                         
    Orientationd(keys=["image"], axcodes="RAS"),
    Spacingd(keys=["image"], pixdim=pixdim, mode="linear"),
    DebugTap("after_spacing")
    ScaleIntensityRangePercentilesd(
    keys=["image"], lower=0.5, upper=99.5, b_min=-1, b_max=1, clip=True)
    DivisiblePadd(
        keys=["image"], k=8,
        mode="constant", constant_values=0
    ),                                                
    ToTensord(keys=["image"]),                      
])

viz_dataset = PersistentDataset(
    data=data_dicts,        
    transform=full_transforms,
    cache_dir="./viz_cache"
)
viz_loader = DataLoader(viz_dataset, batch_size=1, shuffle=False)

fig, axes = plt.subplots(1, 3, figsize=(12, 4))
for ax in axes:
    idx = random.randrange(len(viz_dataset))
    vol = viz_dataset[idx]["image"][0].cpu().numpy()  # [Z, Y, X]
    mid = vol.shape[0] // 2
    ax.imshow(vol[mid], cmap="gray", vmin=0, vmax=1)
    ax.set_title(f"Volume {idx}, slice {mid}")
    ax.axis("off")

plt.tight_layout()
plt.show()


### Define network, scheduler, optimizer, and inferer

We will use a DDPM in this example; for that, we need to define a `DiffusionModelUNet` network that will have as input the noisy images and the values for the timestep `t`, and it will predict the noise that is present in the image.

In this example, we have a network with three levels (with 256, 256, and 512 channels in each). In every level, we will have two residual blocks, and only the last one will have an attention block with a single attention head (with 512 channels).

In [ ]:
device = torch.device("cuda")

model = DiffusionModelUNet(
    spatial_dims=3,
    in_channels=1, out_channels=1,
    num_channels=[32, 64, 128],      # 3 levels
    attention_levels=[False, False, False],
    num_head_channels=[0, 64, 128],
    num_res_blocks=2,
).to(device)

DivisiblePadd(keys=["image"], k=16, mode="constant", constant_values=0)

Together with our U-net, we need to define the Noise Scheduler for the diffusion model. This scheduler is responsible for defining the amount of noise that should be added in each timestep `t` of the diffusion model's Markov chain. Besides that, it has the operations to perform the reverse process, which will remove the noise of the images (a.k.a. denoising process). In this case, we are using a `DDPMScheduler`. Here we are using 1000 timesteps and a `scaled_linear` profile for the beta values (proposed in [Rombach et al. "High-Resolution Image Synthesis with Latent Diffusion Models"](https://arxiv.org/abs/2112.10752)). This profile had better results than the `linear, proposed in the original DDPM's paper. In `beta_start` and `beta_end`, we define the limits for the beta values. These are important to determine how accentuated is the addition of noise in the image.

In [ ]:
scheduler = DDPMScheduler(num_train_timesteps=1000, schedule="scaled_linear_beta", beta_start=0.0005, beta_end=0.0195)

In [ ]:
plt.plot(scheduler.alphas_cumprod.cpu(), color=(2 / 255, 163 / 255, 163 / 255), linewidth=2)
plt.xlabel("Timestep [t]")
plt.ylabel("alpha cumprod")

Finally, we define the Inferer, which contains functions that will help during the training and sampling of the model, and the optimizer.

In [ ]:
inferer = DiffusionInferer(scheduler)

optimizer = torch.optim.Adam(params=model.parameters(), lr=5e-5)

## Model training

In this part, we will train the diffusion model to predict the noise added to the images. For this, we are using an MSE loss between the prediction and the original noise. During the training, we are also sampling brain images to evaluate the evolution of the model. In this training, we use Automatic Mixed Precision to save memory and speed up the training.

In [ ]:
n_epochs = 150
val_interval = 25
epoch_loss_list = []
val_epoch_loss_list = []

scaler = GradScaler()
total_start = time.time()
for epoch in range(n_epochs):
    model.train()
    epoch_loss = 0
    progress_bar = tqdm(enumerate(train_loader), total=len(train_loader), ncols=70)
    progress_bar.set_description(f"Epoch {epoch}")
    for step, batch in progress_bar:
        images = batch["image"].to(device)
        optimizer.zero_grad(set_to_none=True)

        with autocast(device_type="cuda"):
            # Generate random noise
            noise = torch.randn_like(images).to(device)

            # Create timesteps
            timesteps = torch.randint(
                0, inferer.scheduler.num_train_timesteps, (images.shape[0],), device=images.device
            ).long()

            # Get model prediction
            noise_pred = inferer(inputs=images, diffusion_model=model, noise=noise, timesteps=timesteps)

            loss = F.mse_loss(noise_pred.float(), noise.float())

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        epoch_loss += loss.item()

        progress_bar.set_postfix({"loss": epoch_loss / (step + 1)})
    epoch_loss_list.append(epoch_loss / (step + 1))

    if (epoch + 1) % val_interval == 0:
        model.eval()
        val_epoch_loss = 0
        for step, batch in enumerate(val_loader):
            images = batch["image"].to(device)
            noise = torch.randn_like(images).to(device)
            with torch.no_grad():
                with autocast(device_type="cuda"):
                    timesteps = torch.randint(
                        0, inferer.scheduler.num_train_timesteps, (images.shape[0],), device=images.device
                    ).long()

                    # Get model prediction
                    noise_pred = inferer(inputs=images, diffusion_model=model, noise=noise, timesteps=timesteps)
                    val_loss = F.mse_loss(noise_pred.float(), noise.float())

            val_epoch_loss += val_loss.item()
            progress_bar.set_postfix({"val_loss": val_epoch_loss / (step + 1)})
        val_epoch_loss_list.append(val_epoch_loss / (step + 1))

        # Sampling image during training
        image = torch.randn((1, 1, 32, 40, 32))
        image = image.to(device)
        scheduler.set_timesteps(num_inference_steps=1000)
        with autocast(device_type="cuda"):
            image = inferer.sample(input_noise=image, diffusion_model=model, scheduler=scheduler)

        plt.figure(figsize=(2, 2))
        plt.imshow(image[0, 0, :, :, 15].cpu(), vmin=0, vmax=1, cmap="gray")
        plt.tight_layout()
        plt.axis("off")
        plt.show()

total_time = time.time() - total_start
print(f"train completed, total time: {total_time}.")

vae.eval()
x = train_ds[0]["image"].unsqueeze(0).to(device)   # original patch
with torch.no_grad():
    recon, _ = vae(x)

def show_slice(vol, title="", rescale=True):
    """
    Display the middle axial slice of a (Z,Y,X) volume.
    If `rescale=True` the data are mapped to [0,1] just for plotting.
    """
    if torch.is_tensor(vol):
        vol = vol.detach().cpu()
    vol = vol.numpy()           

    z_mid      = vol.shape[0] // 2
    slice_img  = vol[z_mid]      

    if rescale:                 
        slice_img = torch.tanh(torch.as_tensor(slice_img))
        slice_img = (slice_img + 1) / 2                     
        slice_img = slice_img.numpy()

    plt.figure(figsize=(4,4))
    plt.imshow(slice_img, cmap="gray", vmin=0, vmax=1)
    plt.title(title); plt.axis("off"); plt.show()
vol = recon[0, 0]    # now vol.shape == (Z, Y, X)
show_slice(x[0,0], "Input patch")
show_slice(vol, "VAE recon")

### Learning curves

In [ ]:
plt.style.use("seaborn-v0_8")
plt.title("Learning Curves", fontsize=20)
plt.plot(np.linspace(1, n_epochs, n_epochs), epoch_loss_list, color="C0", linewidth=2.0, label="Train")
plt.plot(
    np.linspace(val_interval, n_epochs, int(n_epochs / val_interval)),
    val_epoch_loss_list,
    color="C1",
    linewidth=2.0,
    label="Validation",
)
plt.yticks(fontsize=12)
plt.xticks(fontsize=12)
plt.xlabel("Epochs", fontsize=16)
plt.ylabel("Loss", fontsize=16)
plt.legend(prop={"size": 14})
plt.show()

## Sampling Brain Image

In order to sample the brain images, we need to pass the model an image containing just noise and use it to remove the noise of the image iteratively. For that, we will use the `.sample()` function of the `inferer`.

In [ ]:
ddpm.eval()
vae.eval()

latent_noise = torch.randn((1, 32, 8, 16, 16), device=device)
scheduler.set_timesteps(num_inference_steps=250)

with torch.no_grad(), autocast(device_type="cuda"):
    latent_denoised = inferer.sample(
        input_noise      = latent_noise,
        diffusion_model  = ddpm,
        scheduler        = scheduler
    )

    patch = vae.decode(latent_denoised)  

import matplotlib.pyplot as plt
mid_z = patch.shape[2] // 2
plt.imshow(patch[0, 0, mid_z].cpu(), cmap="gray")
plt.axis("off")
plt.title(f"Synthetic patch (slice {mid_z})")
plt.show()


### Sampling with Denoising Diffusion Implicit Model Scheduler

Recent papers have proposed different ways to improve the sampling speed by using fewer steps in the denoising process. In this example, we are using a `DDIMScheduler` (from [Song et al. "Denoising Diffusion Implicit Models"](https://arxiv.org/abs/2010.02502)) to reduce the original number of steps from 1000 to 250.

In [ ]:
from generative.networks.schedulers import DDIMScheduler

scheduler_ddim = DDIMScheduler(
    num_train_timesteps = 1000,
    schedule            = "scaled_linear_beta",
    beta_start          = 0.0005,
    beta_end            = 0.0195,
    clip_sample         = False,                  
)

scheduler_ddim.set_timesteps(num_inference_steps = 250)
ddpm.eval()
vae.eval()
latent_noise = torch.randn((1, 32, 8, 16, 16), device=device)

with torch.no_grad(), autocast(device_type="cuda"):
    latent_denoised = inferer.sample(
        input_noise     = latent_noise,
        diffusion_model = ddpm,
        scheduler       = scheduler_ddim,
    )
    patch = vae.decode(latent_denoised

import matplotlib.pyplot as plt
mid_z = patch.shape[2] // 2
plt.imshow(patch[0, 0, mid_z].cpu(), cmap="gray")
plt.axis("off")
plt.title(f"DDIM sample (slice {mid_z})")
plt.show()